In [1]:
# run this cell to download data!

import os

directory_path = "data/00--raw/macro"

if not os.path.exists(directory_path) or not os.listdir(directory_path):
    !python3 -m src.fetch.pipeline


2025-05-21 00:47:48,507 - [pipeline.py    ] - [START]   - Running build_indicators_json.py...
2025-05-21 00:47:48,510 - [build_indicator] - [SUCCESS] - indicators.json saved to: data/10--metadata
2025-05-21 00:47:48,510 - [pipeline.py    ] - [SUCCESS] - build_indicators_json.py completed successfully.
2025-05-21 00:47:48,510 - [pipeline.py    ] - [START]   - Running build_countries_json.py...
2025-05-21 00:47:48,512 - [build_countries] - [SUCCESS] - countries.json saved to: data/10--metadata
2025-05-21 00:47:48,512 - [pipeline.py    ] - [SUCCESS] - build_countries_json.py completed successfully.
2025-05-21 00:47:48,512 - [pipeline.py    ] - [START]   - Running build_categories_json.py...
2025-05-21 00:47:48,513 - [build_categorie] - [SUCCESS] - categories.json saved to: data/10--metadata
2025-05-21 00:47:48,514 - [pipeline.py    ] - [SUCCESS] - build_categories_json.py completed successfully.
2025-05-21 00:47:48,514 - [pipeline.py    ] - [START]   - Running download_macro.py...
2025-05

In [2]:
import pandas as pd
from typing import *
from src.preprocess.dataset import Dataset, DatasetConfig
from src.preprocess.result import ResultData


# Usage Example:

dataset_names: Optional[List[str]]

result_data = ResultData(
    datadict = True, # Optional[Dict[str, pd.DataFrame]]
    ml_ready = True, # Optional[pd.DataFrame]
    metadata = True  # Optional["Metadata"]
    )

dataset = Dataset(DatasetConfig(use_raw=True))

result_data = dataset.get(result_data)
datadict = result_data.datadict

In [3]:
from src.clean.clean import clean_datadict
cleaned_dd = clean_datadict(datadict)


In [4]:
from pathlib import Path

def save_cleaned_datasets(
        result: ResultData, 
        base_dir: str = "data/01--clean/macro"
    )-> None:
    """
    Save cleaned datasets to category-based structure in 01--clean/macro.
    """
    base_dir = Path(base_dir)
    if result.datadict is None:
        raise ValueError("result data must contain a valid datadict")
    if result.metadata.category_dict is None:
        raise ValueError("result data must contain valid metadata.category_dict")
    datadict = result.datadict
    category_dict = result.metadata.category_dict

    for name, df in datadict.items():
        category = category_dict.get(name, "uncategorized")
        folder = base_dir / category
        folder.mkdir(parents=True, exist_ok=True)

        file_path = folder / f"{name}_world_bank.csv"
        df.to_csv(file_path)
        print(f"✅ Saved: {file_path}")



result_data.datadict = cleaned_dd

save_cleaned_datasets(result=result_data, base_dir="data/01--clean/")

✅ Saved: data/01--clean/infrastructure-and-technology/individuals_using_the_internet_percent_of_population_world_bank.csv
✅ Saved: data/01--clean/trade-and-commerce/net_trade_in_goods_and_services_current_usd_world_bank.csv
✅ Saved: data/01--clean/trade-and-commerce/fdi_net_inflows_current_usd_world_bank.csv
✅ Saved: data/01--clean/geographic/area_world_bank.csv
✅ Saved: data/01--clean/education/education_years_world_bank.csv
✅ Saved: data/01--clean/sectoral-performance/services_value_added_percent_of_gdp_world_bank.csv
✅ Saved: data/01--clean/sectoral-performance/manufacturing_value_added_percent_of_gdp_world_bank.csv
✅ Saved: data/01--clean/demography/total_population_world_bank.csv
✅ Saved: data/01--clean/demography/economic_activity_world_bank.csv
✅ Saved: data/01--clean/demography/life_expectancy_at_birth_total_years_world_bank.csv
✅ Saved: data/01--clean/demography/gdp_per_person_employed_constant_2011_ppp_usd_world_bank.csv
✅ Saved: data/01--clean/demography/population_size_worl